In [ ]:
import os
from dotenv import load_dotenv
import json
import random
import matplotlib.pyplot as plt
import numpy as np
import asyncio

from benchmarklib import BenchmarkDatabase
from rbf import RandomBooleanFunctionTrial, RandomBooleanFunction
from benchmarklib.compilers import CompileType, XAGCompiler

from qiskit_ibm_runtime import QiskitRuntimeService

import logging
from typing import Iterable, List, Tuple, Dict, Any, Union, Optional
import qiskit
from qiskit.providers import Backend
from qiskit import QuantumCircuit, transpile
import random

from sqlalchemy import select, func
from sqlalchemy.orm import joinedload

from tweedledum.bool_function_compiler import circuit_input, QuantumCircuitFunction
from tweedledum import BitVec

from benchmarklib import CompileType, BenchmarkDatabase
from benchmarklib import BatchQueue
from benchmarklib.compilers import SynthesisCompiler
from benchmarklib.utils.circuit import compute_circuit_metrics
from benchmarklib.algorithms.prediction import compute_analytic_success_rate_estimate1

from qiskit.transpiler import PassManager
from qiskit_ibm_transpiler.ai.routing import AIRouting
from qiskit_ibm_runtime import QiskitRuntimeService

In [ ]:

 
backend = QiskitRuntimeService().backend("ibm_rensselaer")
ai_passmanager = PassManager(
    [
        AIRouting(
            backend=backend,
            optimization_level=2,
            layout_mode="optimize",
            local_mode=True,
        )
    ]
)


In [ ]:
benchmark_db = BenchmarkDatabase("rbf.db", RandomBooleanFunction, RandomBooleanFunctionTrial)

In [ ]:
trials = benchmark_db.query(
    select(RandomBooleanFunctionTrial)
    .where(RandomBooleanFunctionTrial._circuit_pretranspile_qpy != None, RandomBooleanFunctionTrial._circuit_qpy != None)
    .order_by(func.random())
    .limit(100)
    .options(joinedload(RandomBooleanFunctionTrial.problem))
    )
print(len(trials))

In [ ]:
metrics = []
for trial in trials:
    pretranspile_metrics = compute_circuit_metrics(trial.circuit_pretranspile)
    posttranspile_metrics = compute_circuit_metrics(trial.circuit)
    ai_transpiled_circuit = ai_passmanager.run(trial.circuit_pretranspile)
    ai_transpile_metrics = compute_circuit_metrics(ai_transpiled_circuit)
    metrics.append({
        "pretranspile": pretranspile_metrics,
        "reg_transpile": posttranspile_metrics,
        "reg_success_rate": trial.calculate_success_rate(),
        "reg_success_rate_pred": compute_analytic_success_rate_estimate1(backend, trial.circuit),
        "ai_transpile": ai_transpile_metrics,
        "ai_success_rate_pred": compute_analytic_success_rate_estimate1(backend, ai_transpiled_circuit),
    })

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

########################################################################
# EXPECTED INPUT: a list of dictionaries called "results"
#
# Example:
# results = [
#     {
#         "pretranspile": {...},
#         "posttranspile": {...},
#         "ai_transpile": {...}
#     },
#     ...
# ]
########################################################################

# ---- Helper function: count 2-qubit gates -----------------------------

def count_two_qubit_gates(op_counts):
    two_q_gate_names = ["cx", "ecr", "ccx", "ccrx", "ccrx_o1", "ccrx_o2"]
    return sum(op_counts.get(g, 0) for g in two_q_gate_names)


# ---- Gather metrics across circuits ----------------------------------

depth_reg = []
depth_ai = []
gates_reg = []
gates_ai = []
ratio_depth = []
ratio_gates = []
ratio_2q = []
success_reg = []
success_pred_reg = []
success_pred_ai = []

for entry in metrics:
    post = entry["reg_transpile"]
    ai = entry["ai_transpile"]

    d_post = post["circuit_depth"]
    d_ai = ai["circuit_depth"]
    g_post = post["circuit_num_gates"]
    g_ai = ai["circuit_num_gates"]

    depth_reg.append(d_post)
    depth_ai.append(d_ai)
    gates_reg.append(g_post)
    gates_ai.append(g_ai)
    success_reg.append(entry["reg_success_rate"])
    success_pred_ai.append(entry["ai_success_rate_pred"])
    success_pred_reg.append(entry["reg_success_rate_pred"])

    ratio_depth.append(d_post / d_ai if d_ai != 0 else np.nan)
    ratio_gates.append(g_post / g_ai if g_ai != 0 else np.nan)

    post_2q = count_two_qubit_gates(post["circuit_op_counts"])
    ai_2q = count_two_qubit_gates(ai["circuit_op_counts"])
    ratio_2q.append(post_2q / ai_2q if ai_2q != 0 else np.nan)


# Convert to numpy arrays
depth_reg = np.array(depth_reg)
depth_ai = np.array(depth_ai)
gates_reg = np.array(gates_reg)
gates_ai = np.array(gates_ai)
ratio_depth = np.array(ratio_depth)
ratio_gates = np.array(ratio_gates)
ratio_2q = np.array(ratio_2q)
success_reg = np.array(success_reg)
success_pred_reg = np.array(success_pred_reg)
success_pred_ai = np.array(success_pred_ai)


# ---- Build the 4-panel Combined View ----------------------------------

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
(ax1, ax2), (ax3, ax4) = axes


# =====================
# 1. Scatter: Depth
# =====================
ax1.scatter(depth_reg, depth_ai)
max_depth = max(depth_reg.max(), depth_ai.max())
ax1.plot([0, max_depth], [0, max_depth], linestyle="--", color="gray")
ax1.set_title("Circuit Depth: Reg vs AI")
ax1.set_xscale("log")
ax1.set_yscale("log")
ax1.set_xlabel("Reg depth")
ax1.set_ylabel("AI transpile depth")
ax1.grid(True)


# =====================
# 2. Scatter: Total Gates
# =====================
ax2.scatter(gates_reg, gates_ai)
max_gates = max(gates_reg.max(), gates_ai.max())
ax2.plot([0, max_gates], [0, max_gates], linestyle="--", color="gray")
ax2.set_title("Total Gate Count: Reg vs AI")
ax2.set_xscale("log")
ax2.set_yscale("log")
ax2.set_xlabel("Reg num_gates")
ax2.set_ylabel("AI transpile num_gates")
ax2.grid(True)


# =====================
# 3. Scatter: Success Rate Prediction
# =====================
ax3.scatter(success_reg, success_pred_ai)
ax3.plot([0, 1], [0, 1], linestyle="--", color="gray")
ax3.set_title("Predicted Success Rate: Reg vs AI")
ax3.set_xlabel("Reg success rate")
ax3.set_ylabel("AI transpile (predicted) success rate")
ax3.grid(True)


# =====================
# 4. Summary Table
# =====================
ax4.axis("off")

summary_text = f"""
SUMMARY (across {len(metrics)} circuits)

Median depth ratio    = {np.nanmedian(ratio_depth):.2f}
Mean depth ratio      = {np.nanmean(ratio_depth):.2f}

Median gates ratio    = {np.nanmedian(ratio_gates):.2f}
Mean gates ratio      = {np.nanmean(ratio_gates):.2f}

Median 2Q ratio       = {np.nanmedian(ratio_2q):.2f}
Mean 2Q ratio         = {np.nanmean(ratio_2q):.2f}

AI wins (depth)       = {(depth_ai < depth_reg).sum() / len(depth_reg):.2%}
AI wins (gate count)  = {(gates_ai < gates_reg).sum() / len(gates_reg):.2%}
"""

ax4.text(0.05, 0.95, summary_text, fontsize=12, va="top")


plt.tight_layout()
plt.show()